<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/sir_dem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.integrate import solve_ivp

# Configura Matplotlib para renderização matemática interna (compatível com Colab)
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('mathtext', fontset='cm')

# Método Discreto
def sir_disc(beta, gamma, nu, mu, alpha, N0, I0, R0, dt, T):
    passos = int(T / dt) + 1
    S, I, R, N = np.zeros(passos), np.zeros(passos), np.zeros(passos), np.zeros(passos)
    t = np.linspace(0, T, passos)
    S[0], I[0], R[0], N[0] = max(0, N0 - I0 - R0), I0, R0, S[0] + I[0] + R[0]
    for k in range(passos - 1):
        N[k] = S[k] + I[k] + R[k]
        S[k+1] = S[k] + dt * (nu * N[k] - beta * S[k] * I[k] / N[k] - mu * S[k])
        I[k+1] = I[k] + dt * (beta * S[k] * I[k] / N[k] - gamma * I[k] - (mu + alpha) * I[k])
        R[k+1] = R[k] + dt * (gamma * I[k] - mu * R[k])
        N[k+1] = S[k+1] + I[k+1] + R[k+1]
    return t, S, I, R, N

# Método Contínuo
def sir_diff(t, y, beta, gamma, nu, mu, alpha):
    S, I, R = y
    N = S + I + R
    dS = nu * N - beta * S * I / N - mu * S
    dI = beta * S * I / N - gamma * I - (mu + alpha) * I
    dR = gamma * I - mu * R
    return [dS, dI, dR]

def solve_sir(beta, gamma, nu, mu, alpha, N0, I0, R0, dt, T):
    S0 = max(0, N0 - I0 - R0)
    y0 = [S0, I0, R0]
    t_eval = np.arange(0, T + dt, dt)
    t_eval = t_eval[t_eval <= T]
    sol = solve_ivp(sir_diff, [0, T], y0, args=(beta, gamma, nu, mu, alpha), t_eval=t_eval)
    N = sol.y[0] + sol.y[1] + sol.y[2]
    return sol.t, sol.y[0], sol.y[1], sol.y[2], N

# Widgets para parâmetros interativos (valores iniciais do último script)
beta_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.001, value=0.3, description='beta', readout_format='.3f')
gamma_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.001, value=0.122, description='gamma', readout_format='.3f')
nu_slider = widgets.FloatSlider(min=0.0001, max=0.1, step=0.0001, value=0.0082, description='nu', readout_format='.4f')
mu_slider = widgets.FloatSlider(min=0.0001, max=0.1, step=0.0001, value=0.0072, description='mu', readout_format='.4f')
alpha_slider = widgets.FloatSlider(min=0.0001, max=0.1, step=0.0001, value=0.0420, description='alpha', readout_format='.4f')
N0_slider = widgets.IntSlider(min=100, max=10000, step=10, value=10000, description='N0', readout_format='d')
I0_slider = widgets.IntSlider(min=0, max=500, step=1, value=10, description='I0', readout_format='d')
R0_slider = widgets.IntSlider(min=0, max=500, step=1, value=0, description='R0', readout_format='d')
T_slider = widgets.IntSlider(min=50, max=2000, step=10, value=1825, description='T', readout_format='d')
dt_disc_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.5, description='dt Disc', readout_format='.2f')
dt_cont_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='dt Cont', readout_format='.2f')

# Função de plotagem
def plot_sir(beta, gamma, nu, mu, alpha, N0, I0, R0, dt_disc, dt_cont, T):
    # Simulação Discreta
    t_disc, S_disc, I_disc, R_disc, N_disc = sir_disc(beta, gamma, nu, mu, alpha, N0, I0, R0, dt_disc, T)

    # Simulação Contínua
    t_cont, S_cont, I_cont, R_cont, N_cont = solve_sir(beta, gamma, nu, mu, alpha, N0, I0, R0, dt_cont, T)

    # Gráficos
    plt.figure(figsize=(14, 10), dpi=200)

    # t x I(t) Discreto
    plt.subplot(2, 2, 1)
    plt.plot(t_disc, I_disc, linewidth=0.5, label='Discreto', color='red', linestyle='dashed')
    plt.plot(t_cont, I_cont, linewidth=0.5, label='Contínuo', color='blue')
    plt.xlabel('$t$ (dias)')
    plt.ylabel('$I(t)$')
    plt.legend()
    plt.grid(True)

    # t x N(t) Discreto
    plt.subplot(2, 2, 2)
    plt.plot(t_disc, N_disc, linewidth=0.5, label='Discreto', color='red', linestyle='dashed')
    plt.plot(t_cont, N_cont, linewidth=0.5, label='Contínuo', color='blue')
    plt.xlabel('$t$ (dias)')
    plt.ylabel('$N(t)$')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Interface interativa
ui = widgets.VBox([
    beta_slider, gamma_slider, nu_slider, mu_slider, alpha_slider,
    N0_slider, I0_slider, R0_slider, T_slider, dt_disc_slider, dt_cont_slider
])
out = widgets.interactive_output(plot_sir, {
    'beta': beta_slider, 'gamma': gamma_slider, 'nu': nu_slider, 'mu': mu_slider, 'alpha': alpha_slider,
    'N0': N0_slider, 'I0': I0_slider, 'R0': R0_slider, 'T': T_slider,
    'dt_disc': dt_disc_slider, 'dt_cont': dt_cont_slider
})
display(ui, out)

Output()